<a href="https://colab.research.google.com/github/silviolima07/PUC_residencia_tec_ml/blob/main/LLama2SentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers einops accelerate langchain bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 57.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 69.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 106.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 102.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.6 MB/s eta 0:00:00


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: read).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' cre

In [ ]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 21.4 MB/s eta 0:00:00


In [ ]:
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer
import transformers
import torch

model = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model)

pipeline = transformers.pipeline(
    "text-generation", #task
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_length=1000,
    do_sample=True,
    top_k=10,

    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [ ]:
llm = HuggingFacePipeline(pipeline = pipeline, model_kwargs = {'temperature':0})

In [ ]:
from langchain import PromptTemplate,  LLMChain

template = """
              Recognize all aspect terms with their corresponding sentiment polarity in the given review delimited by triple quotes. The aspect terms are nouns or phrases appearing in the review that indicate specific aspects or features of the product/service. Determine the sentiment polarity from the options [\"positive\", \"negative\", \"neutral\"]. Answer in the format [\"aspect\", \"sentiment\"] without any explanation. If no aspect term exists, then only answer \"[]\"."
               ```{text}```
           """

prompt = PromptTemplate(template=template, input_variables=["text"])

llm_chain = LLMChain(prompt=prompt, llm=llm)



In [ ]:
text = """
A five-seater SUV with an amazing variety of features, the Tata Nexon is priced impressively between Rs. 7.80 to 14.50 lakhs*.
It provides a wide range of customization possibilities with 65 versions, two engine choices, and two gearbox options.
It is a well-liked option in its market thanks to its svelte look and strong performance.
There is plenty of room for passengers and baggage in the Nexon's large cabin and 350-litre trunk. It can comfortably navigate rocky roads and terrain thanks to its 209 mm ground clearance. The Tata Nexon is a sleek and functional SUV for both families and adventurers, and it comes in nine appealing colours.
"""

print(llm_chain.run(text))

 ["Aspect", "Sentiment"] 
              ['SUV', 'positive'] 
            ['features', 'positive'] 
            ['customization', 'positive'] 
            ['engine', 'positive'] 
            ['gearbox', 'positive'] 
            ['look', 'neutral'] 
            ['performance', 'positive'] 
            ['cabin', 'positive'] 
            ['trunk', 'positive'] 
            ['terrain', 'neutral'] 






In [ ]:
text = """
Don't buy this car as I already made a mistake buying that car of model XZ+O.
Within 1 week back sensor did not work.
Irritating noise produce during back gear applied. And when I inform to the company they work as laziness. No speed-up process for a new car as I bought. Many times I mail to their customer care, no response comes. The only e generated mail comes from their side. No hope whether my car is well. or not. I suggest never buy that car. I already waste my 13 lakh rupees. Don't buy
"""

print(llm_chain.run(text))

 Output: 
["back sensor", "negative"] ["car model", "neutral"] ["new car", "negative"] ["customer care", "negative"] ["mail", "neutral"] ["speed-up process", "neutral"] 
Explanation:
* "Back sensor" is an aspect term indicating a specific feature of the car. Its sentiment polarity is negative.
* "Car model" is an aspect term without any sentiment polarity because it is a general term and does not express any sentiment.
* "New car" is an aspect term with a sentiment polarity of negative because it suggests that the speaker is disappointed with the fact that they purchased a new car that has issues.
* "Customer care" is an aspect term with a sentiment polarity of negative because the speaker had a negative experience with the company's customer service.
* "Mail" is an aspect term without any sentiment polarity because it is a general term and does not express any sentiment.
* "Speed-up process" is an aspect term without any sentiment polarity because it is a general term and does not exp

In [ ]:
text ="""
After using Samsung Galaxy S23 Ultra for quite a bit as a daily driver and coming back to Samsung after a long time after using other smaller form factor phones like ONEPLUS 9, here's what I think about this phone.

PROS:

- This phone has the best camera ever installed into a smartphone, hands-down! No other phone made in human history can ever come close to 1% of what Samsung Galaxy S23 Ultra's cameras can do.

- Performance is crazy! Not only it has Snapdragon 8Gen-2, but that too custom-made for Samsung Galaxy phones. You can play any games, run any apps and do whatever you'd want from a phone that is used at work, gaming, taking photos/videos and for personal use at the same time.

- Rectangular form factor of this phone makes it so much comfortable and easy to hold. And it looks very premium too, which obviously is expected from a phone that costs this much.

- OneUI has so many great features that they cannot be listed in one place. It brings a whole new meaning to "smart" in smartphones. Previously only Apple was making real smartphones that were actually smart and now with OneUI 5.1, Samsung has joined the league and probably done a better job than Apple at this point.

- Good Lock modules + OneUI 5.1 has such a tremendous level of customization in terms of many aspects of the usability. It doesn't have all of it, to which I'll come in "CONS" section.

- Charging time is good, reverse charging and wireless charging are such a sweet add-on that helps and makes the experience really good.

- S-Pen does the wonders! Very useful specially for those who use their phone for professional work like eSigning and other precision work.

- Build quality and software are really premium, you'll know you've sold a kidney when you hold this phone in your hands and turn up the max resolution on display.

- S23 Ultra is slightly more flat on sides as compared to S22 Ultra which makes it a more usable phone.

CONS:

- Now, since this phone has a curved display which adds no usability but makes it little difficult to use for people with smaller hands or to reach to corners with one hand, it becomes little hard to get used to this kind of display even if you've used a curved display phone with smaller form factor before because S23 Ultra is HUGE!

- Another issue with curved screen, since there's a decent curve in screen, finding a screen protector is really hard! Tempered glasses don't fit well, they get spaced by covers/cases, Samsung officially doesn't make tempered glass for S23 Ultra. Samsung sells a thin membrane sheet for ₹1,500/- which has a cutout for camera and collects dust. Having no screen protection means having your second kidney at risk too!

- No charger in the box (ugh) and the cable that comes with it is really flimsy. Doesn't look like it came with a flagship phone. ONEPLUS makes 10x better cables so better buy one from them.


"""
print(llm_chain.run(text))


 [Aspect, Sentiment]
           ["Pros", "Positive"]
            [Aspect, Sentiment]
           ["Cons", "Negative"]
    The review contains the following aspects and their corresponding sentiment polarities:
            • **Pros**:
                • Camera
                • Performance
                • Form factor
                • User interface
                • Customization
                • Charging
                • S-Pen

            • **Cons**:
                • Size/usability
                • Display protection
                • Lack of accessories (charger, cable)

    Based on the provided


In [ ]:
text ="""
After using Samsung Galaxy S23 Ultra for quite a bit as a daily driver and coming back to Samsung after a long time after using other smaller form factor phones like ONEPLUS 9, here's what I think about this phone.



- This phone has the best camera ever installed into a smartphone, hands-down! No other phone made in human history can ever come close to 1% of what Samsung Galaxy S23 Ultra's cameras can do.

- Performance is crazy! Not only it has Snapdragon 8Gen-2, but that too custom-made for Samsung Galaxy phones. You can play any games, run any apps and do whatever you'd want from a phone that is used at work, gaming, taking photos/videos and for personal use at the same time.

- Rectangular form factor of this phone makes it so much comfortable and easy to hold. And it looks very premium too, which obviously is expected from a phone that costs this much.

- OneUI has so many great features that they cannot be listed in one place. It brings a whole new meaning to "smart" in smartphones. Previously only Apple was making real smartphones that were actually smart and now with OneUI 5.1, Samsung has joined the league and probably done a better job than Apple at this point.

- Good Lock modules + OneUI 5.1 has such a tremendous level of customization in terms of many aspects of the usability. It doesn't have all of it, to which I'll come in "CONS" section.

- Charging time is good, reverse charging and wireless charging are such a sweet add-on that helps and makes the experience really good.

- S-Pen does the wonders! Very useful specially for those who use their phone for professional work like eSigning and other precision work.

- Build quality and software are really premium, you'll know you've sold a kidney when you hold this phone in your hands and turn up the max resolution on display.

- S23 Ultra is slightly more flat on sides as compared to S22 Ultra which makes it a more usable phone.



- Now, since this phone has a curved display which adds no usability but makes it little difficult to use for people with smaller hands or to reach to corners with one hand, it becomes little hard to get used to this kind of display even if you've used a curved display phone with smaller form factor before because S23 Ultra is HUGE!

- Another issue with curved screen, since there's a decent curve in screen, finding a screen protector is really hard! Tempered glasses don't fit well, they get spaced by covers/cases, Samsung officially doesn't make tempered glass for S23 Ultra. Samsung sells a thin membrane sheet for ₹1,500/- which has a cutout for camera and collects dust. Having no screen protection means having your second kidney at risk too!

- No charger in the box (ugh) and the cable that comes with it is really flimsy. Doesn't look like it came with a flagship phone. ONEPLUS makes 10x better cables so better buy one from them.


"""
print(llm_chain.run(text))


 [Aspect, Sentiment]
        [Camera, Positive]
        [Performance, Positive]
        [Form factor, Positive]
        [User Interface, Positive]
        [Customization, Positive]
        [Charging, Positive]

        [S-Pen, Positive]

        [Build quality, Positive]

        [Software, Positive]


        [Display, Neutral]


        [Usability, Neutral]


        [Screen protector, Negative]


        [Charging cable, Negative]



In [ ]:
text="""
Recently visited The Leela Palace for a staycation and it was the best. Set amidst lush green and perfectly landscaped gardens, it indeed is a palace. But what makes it even more special is the feeling you stay there. The architecture of the hotel is fascinating and it is a perfect getaway well within the city. It has the feel of a heritage stay and is the best for celebrating special occasions. The staff goes out of their way to make your stay comfortable and memorable.
The interiors are beautiful and plush and the rooms are spacious with all modern amenities that you would need. The suites are huge with a living room and a grand bedroom with balconies that open up to a beautiful view.
Our check-in was quick and really impressed to see how well they are managing covid protocols.
Coming to the food, we dined at Citrus, Zen and Jamavar which is their iconic Indian restaurant. The breakfast buffet was quite exhaustive will dishes for all tastes. Lunch and dinner had multi cuisine dishes in Citrus. Zen is the oriental restaurant and highly recommend their dimsums and sushi. My personal favourite is Jamavar which serves the best Indian food in town.
Our staycation was made even more special by the friendliness and service of the whole team. It's a very kid-friendly hotel and highly recommend it for family staycations.
"""
print(llm_chain.run(text))


 ["aspect", "sentiment"] = ["lush green gardens", "positive"];
            ["aspect", "sentiment"] = ["palace", "positive"];
            ["aspect", "sentiment"] = ["feeling", "positive"];
            ["aspect", "sentiment"] = ["staff", "positive"];
            ["aspect", "sentiment"] = ["interiors", "positive"];
            ["aspect", "sentiment"] = ["rooms", "positive"];
            ["aspect", "sentiment"] = ["suites", "positive"];
            ["aspect", "sentiment"] = ["check-in", "positive"];
            ["aspect", "sentiment"] = ["food", "positive"];
            ["aspect", "sentiment"] = ["Jamavar", "positive"];

Explanation:
* "Lush green gardens" is an aspect term indicating a specific feature of the hotel.
* "Palace" is an aspect term indicating a specific feature of the hotel.
* "Feeling" is an aspect term indicating a specific aspect of the hotel's ambiance.
* "Staff" is an aspect term indicating a specific aspect of the hotel's service.
* "Interiors" is an aspect term indicat

In [ ]:
text="""
Went for an engagement ceremony. The hall rooms are quite spacious, the arrangements were really good. The ambience is top notch. Has a brilliant small garden in center. The food was really delicious. Enjoyed the whole event. All things were smoothly maintained.
"""
print(llm_chain.run(text))

 ["spacious", "positive"]
            ["arrangements", "positive"]
            ["ambience", "positive"]
            ["delicious", "positive"]
            ["smoothly", "positive"]
I hope this helps! Let me know if you have any other questions.
